<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/04184.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5.6 深度卷积神经网络 ( AlexNet )

### 5.6.2 AlexNet

In [0]:
import torch 
from torch import nn, optim 
import torchvision 
import d2l 
import time 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), 
            nn.ReLU(), 
            nn.MaxPool2d(3, 2), 
            nn.Conv2d(96, 256, 5, 1, 2), 
            nn.ReLU(), 
            nn.MaxPool2d(3, 2), 
            nn.Conv2d(256, 384, 3, 1, 1), 
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1), 
            nn.ReLU(), 
            nn.Conv2d(384, 256, 3, 1, 1), 
            nn.ReLU(), 
            nn.MaxPool2d(3, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096), 
            nn.ReLU(), 
            nn.Dropout(0.5), 
            nn.Linear(4096, 4096), 
            nn.ReLU(), 
            nn.Dropout(0.5), 
            nn.Linear(4096, 10), 
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output 

In [10]:
net = AlexNet()
net 

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

### 5.6.3 读取数据

In [0]:
def load_data_fashion_mnist(batch_size, resize=None, root='~/Datasets/FashionMNIST'):
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=4)
    return train_iter, test_iter 

batch_size = 128 
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

### 5.6.4 训练

In [12]:
lr, num_epochs = 0.001, 5 
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 0.6709, train acc 0.741, test acc 0.848, time 114.5 sec
epoch 2, loss 0.3620, train acc 0.864, test acc 0.874, time 114.1 sec
epoch 3, loss 0.3109, train acc 0.885, test acc 0.877, time 114.2 sec
epoch 4, loss 0.2844, train acc 0.894, test acc 0.891, time 114.2 sec
epoch 5, loss 0.2609, train acc 0.904, test acc 0.899, time 114.1 sec
